In [1]:
!pip install anvil-uplink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.1 MB/s eta 0:00:00


In [ ]:
import asyncio
import nest_asyncio
from uagents import Agent, Context, Model
import os
from groq import Groq

# Apply nest_asyncio to avoid event loop conflicts
nest_asyncio.apply()

# Define the message model
class BudgetQuery(Model):
    query: str

# Define the agent
budget_agent = Agent(name="BudgetAssistant")

# Queue to store agent responses
agent_output_queue = asyncio.Queue()

@budget_agent.on_message(model=BudgetQuery)
async def handle_message(ctx: Context, message: BudgetQuery):
    key = os.getenv("GROQ_API_KEY")  # Ensure API key is stored in environment variables
    client = Groq(api_key=key)

    similar_chunks = vector_store.similarity_search(message.query, n_results=5)
    context = "\n".join([chunk.content for chunk in similar_chunks])

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a financial assistant providing detailed budget summaries based on user data."},
            {"role": "user", "content": f"Here is the context and the query which is to be answered using the context:\n{context}\n\nNow, {message.query}"},
        ],
        model="llama-3.3-70b-versatile",
    )

    response = chat_completion.choices[0].message.content
    await agent_output_queue.put(response)  # Store response in the queue
    ctx.send(response)

async def get_budget_response(query):
    message = BudgetQuery(query=query)
    await budget_agent.put(message)  # Send message to agent
    return await agent_output_queue.get()  # Wait for response

async def main():
    asyncio.create_task(budget_agent.run_async())

if __name__ == "__main__":
    asyncio.run(main())


In [2]:
import anvil.server
import matplotlib.pyplot as plt
import io
import base64
import re
import anvil.server
import json

In [ ]:
anvil.server.disconnect()
anvil.server.connect("INSERT YOUR ANVIL SERVER KEY HERE")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [29]:
import json
import anvil.server

@anvil.server.callable
def orc(text):

    print(f"Received text: {text}")
    classified = OrchestrationAgent(text)
    print(f"Classified as: {classified}")

    response_data = {"classification": classified, "response": "", "extra_info": ""}

    if classified == "BudgetBreakDownAgent":
        response = BudgetBreakDownAgent(text)
        print(f"Budget Breakdown Response: {response}")

        # GraphPlottingAgent(response)
        items = CategoryWiseBreakdownAgent(response)
        print(f"Category Breakdown Items: {items}")

        if items:
            button = items[0]
            CategoryAnalysis = CategoryBudgetAnalysisAgent(button)
            print(f"Category Analysis: {CategoryAnalysis}")
            return CategoryAnalysis
            response_data["response"] = str(CategoryAnalysis)
        else:
            return response
            response_data["response"] = str(response)

    elif classified == "EssentialExpenditureAgent":
        response = EssentialExpenditureAgent(text)
        print(f"Essential Expenditure Response: {response}")
        return response

        GraphPlottingAgent(response)
        response1 = FinancialAdvisorAgent(response)
        print(f"Financial Advisor Response: {response1}")
        # GraphPlottingAgent(response1)
        return response1

        response_data["response"] = str(response1)

    elif classified == "FinancialPlanningAgent":
        response = EssentialExpenditureAgent(text)
        response1 = FinancialAdvisorAgent(response)
        response2 = FinancialPlanningAgent(text, response1)

        print(f"Financial Planning Response: {response2}")
        return response2

        # GraphPlottingAgent(response2)

        response_data["response"] = str(response2)

    else:
        response_data["response"] = "Invalid Classification"

    # Convert to JSON string for better readability in Anvil’s text box
    json_string=json.dumps(response_data, indent=4)

    return 'Invalid Classification'











@anvil.server.callable
def generate_graph(context, graph_type="auto"):
    """
    Generate a graph visualization based on financial data in the context.

    Args:
        context (str): Text containing financial information
        graph_type (str): Type of graph ('bar', 'pie', 'line', or 'auto')

    Returns:
        str: Base64 encoded image for Anvil image placeholder
    """
    # Extract financial data from the context
    expenses = extract_financial_data(context)

    if not expenses:
        return create_error_graph("No financial data found in the provided context")

    # Determine best graph type if set to auto
    if graph_type == "auto":
        if "month" in context.lower() or "monthly" in context.lower():
            graph_type = "bar" if len(expenses) > 6 else "pie"
        elif "category" in context.lower():
            graph_type = "pie"
        elif "trend" in context.lower() or "over time" in context.lower():
            graph_type = "line"
        else:
            graph_type = "bar" if len(expenses) > 5 else "pie"

    # Create appropriate visualization
    if graph_type == "pie":
        return create_pie_chart(expenses)
    elif graph_type == "line":
        return create_line_chart(expenses)
    else:  # Default to bar
        return create_bar_chart(expenses)

def extract_financial_data(context):
    """Extract financial data from context string"""
    expenses = {}

    # Look for common expense patterns in financial text
    # Pattern for category and amount (e.g., "Groceries: $250")
    category_pattern = r'([A-Za-z\s&]+):\s*\$?(\d+,?\d*(?:\.\d+)?)'
    matches = re.findall(category_pattern, context)

    for category, amount in matches:
        category = category.strip()
        # Convert amount string to float, handling commas
        amount_value = float(amount.replace(',', ''))
        if category in expenses:
            expenses[category] += amount_value
        else:
            expenses[category] = amount_value

    # If no matches found, try simpler pattern
    if not expenses:
        lines = context.split('\n')
        for line in lines:
            # Look for lines with dollar amounts
            amount_match = re.search(r'\$(\d+,?\d*(?:\.\d+)?)', line)
            if amount_match:
                amount = float(amount_match.group(1).replace(',', ''))
                # Extract category (text before the amount)
                category_match = re.search(r'^([A-Za-z\s&]+)', line)
                if category_match:
                    category = category_match.group(1).strip()
                    if len(category) > 2 and category.lower() not in ['total', 'sum']:
                        expenses[category] = amount

    return expenses


@anvil.server.callable
def generate_secondary_graph(context, graph_type="auto"):
    """
    Generate a secondary graph visualization that's different from the primary one.

    Args:
        context (str): Text containing financial information
        primary_graph_type (str): Type of the primary graph ('bar', 'pie', 'line')

    Returns:
        str: Base64 encoded image for Anvil image placeholder


    """
    expenses = extract_financial_data(context)

    if graph_type == "auto":
        if "month" in context.lower() or "monthly" in context.lower():
            graph_type = "bar" if len(expenses) > 6 else "pie"
        elif "category" in context.lower():
            graph_type = "pie"
        elif "trend" in context.lower() or "over time" in context.lower():
            graph_type = "line"
        else:
            graph_type = "bar" if len(expenses) > 5 else "pie"
    primary_graph_type = graph_type
    # Extract financial data from the context


    if not expenses:
        return create_error_graph("No financial data found in the provided context")

    # Determine secondary graph type that complements the primary
    if primary_graph_type == "pie":
        # If primary is pie, show bar or line for different perspective
        if "trend" in context.lower() or "over time" in context.lower():
            secondary_type = "line"
        else:
            secondary_type = "bar"
    elif primary_graph_type == "bar":
        # If primary is bar, show pie or stacked bar
        if len(expenses) <= 8:
            secondary_type = "pie"
        else:
            secondary_type = "stacked_bar"
    elif primary_graph_type == "line":
        # If primary is line, show bar or area chart
        secondary_type = "bar"
    else:
        # Default fallback
        secondary_type = "pie" if primary_graph_type == "bar" else "bar"

    # Create appropriate visualization
    if secondary_type == "pie":
        return create_pie_chart(expenses)
    elif secondary_type == "line":
        return create_line_chart(expenses)
    elif secondary_type == "stacked_bar":
        return create_stacked_bar_chart(expenses)
    else:  # Default to bar
        return create_bar_chart(expenses)

def create_stacked_bar_chart(data):
    """Create a stacked bar chart showing spending levels"""
    plt.figure(figsize=(10, 6))

    categories = list(data.keys())
    values = list(data.values())

    # Define spending levels (low, medium, high)
    max_val = max(values)
    high_thresh = max_val * 0.6
    medium_thresh = max_val * 0.3

    # Create stacked bars
    low = [min(v, medium_thresh) for v in values]
    medium = [min(max(v - medium_thresh, 0), high_thresh - medium_thresh) for v in values]
    high = [max(v - high_thresh, 0) for v in values]

    plt.bar(categories, low, color='lightgreen', label='Low')
    plt.bar(categories, medium, bottom=low, color='gold', label='Medium')
    plt.bar(categories, high, bottom=[l+m for l,m in zip(low, medium)], color='salmon', label='High')

    plt.xlabel('Category')
    plt.ylabel('Amount ($)')
    plt.title('Spending by Level')
    plt.xticks(rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()

    return fig_to_base64()

def create_bar_chart(data):
    """Create a bar chart from the data"""
    plt.figure(figsize=(10, 6))
    bars = plt.bar(data.keys(), data.values(), color='skyblue')
    plt.xlabel('Category')
    plt.ylabel('Amount ($)')
    plt.title('Expense Breakdown')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'${int(height)}', ha='center', va='bottom')

    return fig_to_base64()

def create_pie_chart(data):
    """Create a pie chart from the data"""
    plt.figure(figsize=(10, 7))
    wedges, texts, autotexts = plt.pie(
        data.values(),
        labels=data.keys(),
        autopct='%1.1f%%',
        startangle=90,
        shadow=True
    )

    # Improve text readability
    for text in texts:
        text.set_fontsize(9)
    for autotext in autotexts:
        autotext.set_fontsize(9)
        autotext.set_color('white')

    plt.axis('equal')
    plt.title('Expense Distribution')
    plt.tight_layout()

    return fig_to_base64()

def create_line_chart(data):
    """Create a line chart showing trends"""
    plt.figure(figsize=(10, 6))
    plt.plot(list(data.keys()), list(data.values()), marker='o', linestyle='-', color='blue')
    plt.xlabel('Category/Period')
    plt.ylabel('Amount ($)')
    plt.title('Expense Trend')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, linestyle='--', alpha=0.7)

    # Add value labels
    for i, val in enumerate(data.values()):
        plt.text(i, val + max(data.values())*0.02, f'${int(val)}', ha='center')

    plt.tight_layout()

    return fig_to_base64()

def create_error_graph(message):
    """Create a simple graph with an error message"""
    plt.figure(figsize=(8, 6))
    plt.text(0.5, 0.5, message, ha='center', va='center', fontsize=14)
    plt.axis('off')
    return fig_to_base64()

def fig_to_base64():
    """Convert matplotlib figure to base64 string for Anvil"""
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=100)
    buf.seek(0)
    img_str = base64.b64encode(buf.getvalue()).decode('utf-8')
    plt.close()
    return img_str





In [5]:
!pip install groq

!pip install -qU langchain-community pathway pyngrok sentence_transformers unstructured unstructured[pdf] pdfplumber pymupdf Pillow pandas tiktoken llm-app

!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
from typing import List, Dict, Optional
from dataclasses import dataclass
import pickle
import os
import pathway as pw
# from pathway.xpacks.llm.vector_store   import VectorStoreServer
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
import faiss
from pyngrok import ngrok

@dataclass
class Document:
    id: str
    content: str
    metadata: Optional[Dict] = None

@dataclass
class Chunk:
    id: str
    doc_id: str
    content: str
    vector: np.ndarray = None
    metadata: Optional[Dict] = None

class EnhancedVectorStore:
    def __init__(
        self,
        embedding_model: str = "nomic-ai/nomic-embed-text-v1",
        chunk_size: int = 500,
        host: str = "127.0.0.1",
        port: int = 8668,
        base_path: str = "vector_db",
        cache_dir: str = "./Cache",
        license_key: str = None
    ):
        self.embedding_model = embedding_model
        self.chunk_size = chunk_size
        self.host = host
        self.port = port
        self.base_path = base_path
        self.cache_dir = cache_dir

        if license_key:
            pw.set_license_key(license_key)

        self.embeddings = SentenceTransformerEmbeddings(
            model_name=embedding_model,
            model_kwargs={"trust_remote_code": True}
        )
        self.text_splitter = CharacterTextSplitter(chunk_size=chunk_size)

        os.makedirs(base_path, exist_ok=True)
        os.makedirs(cache_dir, exist_ok=True)

    def process_documents(self, documents: List[Document]) -> List[Chunk]:
        """Process documents into chunks with embeddings."""
        chunks = []
        for doc in documents:
            texts = self.text_splitter.split_text(doc.content)
            for i, text in enumerate(texts):
                chunk = Chunk(
                    id=f"{doc.id}chunk{i}",
                    doc_id=doc.id,
                    content=text,
                    metadata=doc.metadata
                )
                chunk.vector = self.embeddings.embed_query(text)
                chunks.append(chunk)
        return chunks

    def store_chunks(self, chunks: List[Chunk]):
        """Store chunks directly in the vector database using FAISS."""
        # Check if chunks is empty before accessing elements
        if not chunks:
            print("No chunks to store.")
            return

        index = faiss.IndexFlatL2(len(chunks[0].vector))
        vectors = np.array([chunk.vector for chunk in chunks])
        index.add(vectors)

        db_path = os.path.join(self.base_path, "chunks_db.index")
        faiss.write_index(index, db_path)

        chunk_metadata = {chunk.id: chunk for chunk in chunks}
        with open(os.path.join(self.base_path, "chunks_metadata.pkl"), "wb") as f:
            pickle.dump(chunk_metadata, f)

    def similarity_search(self, query: str, n_results: int = 5) -> List[Chunk]:
        """Perform similarity search directly on chunks."""
        query_vector = self.embeddings.embed_query(query)
        db_path = os.path.join(self.base_path, "chunks_db.index")
        index = faiss.read_index(db_path)

        with open(os.path.join(self.base_path, "chunks_metadata.pkl"), "rb") as f:
            chunk_metadata = pickle.load(f)

        D, I = index.search(np.array([query_vector]), n_results)
        results = [chunk_metadata[list(chunk_metadata.keys())[idx]] for idx in I[0]]
        return results

def run_server(self):
        """Run a Pathway web server to accept query and context dynamically."""
        webserver = pw.io.http.PathwayWebserver(host=self.host, port=self.port, with_cors=True)

        def serve(route, schema, handler, documentation):
            queries, writer = pw.io.http.rest_connector(
                webserver=webserver,
                route=route,
                methods=("POST",),
                schema=schema,
                autocommit_duration_ms=50,
                delete_completed_queries=False,
                documentation=documentation,
            )
            writer(handler(queries))

        class QuerySchema(pw.Schema):
            query: str
            context: str

        def process_query(queries):
            """Process user query and store context before retrieving results."""
            for query in queries:
                context_doc = Document(id=f"ctx_{query['query']}", content=query['context'])
                chunks = self.process_documents([context_doc])
                self.store_chunks(chunks)
                return self.similarity_search(query['query'])

        serve(
            "/v1/retrieve",
            QuerySchema,
            process_query,
            pw.io.http.EndpointDocumentation(
                summary="Store context and retrieve similar documents",
                description="Stores provided context in vector store and retrieves similar documents based on query.",
                method_types=("POST",),
            ),
        )

        webserver._run()

vector_store = EnhancedVectorStore()
documents = [
    Document(id="doc1", content="01-Feb-2025, Rent, 8000"),
    Document(id="doc2", content="02-Feb-2025, Groceries, 2500"),
    Document(id="doc3", content="03-Feb-2025, Electricity Bill, 1200"),
    Document(id="doc4", content="04-Feb-2025, Internet Bill, 800"),
    Document(id="doc5", content="05-Feb-2025, Transportation, 1500"),
    Document(id="doc6", content="06-March-2025, Dining Out, 900"),
    Document(id="doc7", content="07-Feb-2025, Mobile Recharge, 500"),
    Document(id="doc8", content="08-March-2025, Gym Membership, 2000"),
    Document(id="doc9", content="09-Feb-2025, Stationery, 400"),
    Document(id="doc10", content="10-March-2025, Movie Ticket, 600"),
    Document(id="doc11", content="11-Feb-2025, Medicines, 1100"),
    Document(id="doc12", content="12-April-2025, Books, 1800"),
    Document(id="doc13", content="13-April-2025, Subscription Services, 1200"),
    Document(id="doc14", content="14-Feb-2025, Valentine's Gift, 2500"),
    Document(id="doc15", content="15-March-2025, Travel, 3000"),
    Document(id="doc16", content="16-Feb-2025, Groceries, 2200"),
    Document(id="doc17", content="17-Feb-2025, Car Fuel, 2800"),
    Document(id="doc18", content="18-Feb-2025, Home Maintenance, 1500"),
    Document(id="doc19", content="19-March-2025, Clothing, 3500"),
    Document(id="doc20", content="20-Feb-2025, Coffee, 300"),
]

chunks = vector_store.process_documents(documents)
vector_store.store_chunks(chunks)


key = "INSERT YOUR GROK API KEY HERE"

import os
from groq import Groq
client = Groq(
    api_key=key,
)
# def BudgetBreakDownAgent(query):
#   # query = "Give a detailed description of my budget expenses in the month of March."
#   similar_chunks = vector_store.similarity_search(query, n_results=5)

#   context = "\n".join([chunk.content for chunk in similar_chunks])

#   chat_completion = client.chat.completions.create(
#       messages=[
#           {"role": "system", "content": "You are a financial assistant providing detailed budget summaries based on user data."},
#           {"role": "user", "content": f"Here are is the context and the query which is to be answered using the context:\n{context}\n\nNow, {query}."},
#       ],
#       model="llama-3.3-70b-versatile",
#   )

#   response = chat_completion.choices[0].message.content
#   return response
import subprocess

def BudgetBreakDownAgent(query):
    # Run the external script
    subprocess.run(["python", "/content/budget_breakdown.py", query])

    # Read the generated response
    with open("/content/budget_summary.txt", "r") as f:
        response = f.read()
    print(response)

    return str(response)



def GraphPlottingAgent(context):
  # query1 = response

  chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are a data visualization assistant specializing in financial data representation. Generate a bar graph or a pie chart to visualise the data provided in the query"},
          {"role": "user", "content": f"Generate a interactive and beautiful Python code to visualize the following financial data. Do not provide any description of the code generated, output should be only the code:\n\n{context}"},
      ],
      model="llama-3.3-70b-versatile",
  )

  code = chat_completion.choices[0].message.content
  code = code.lstrip("python")
  code = code.rstrip("")
  exec(code, globals())

def CategoryWiseBreakdownAgent(context):
  chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are a financial advisor agent providing a list of items on which user has major expenditure."},
          {"role": "user", "content": f"Given a context input:\n{context}, return a list of items of which there is major expenditure for the user. Output format:PYTHON LIST. Do not give any explanation or description"},
      ],
      model="llama-3.3-70b-versatile",
  )

  items = chat_completion.choices[0].message.content
  # items = CategoryWiseBreakdownAgent(response)
  items = items.lstrip("python")
  items = items.rstrip("")

  return items

button = 'Clothing'

def CategoryBudgetAnalysisAgent(button):
  query111 = f'Give a detailed description of my budget expenses in {button}.'
  similar_chunks = vector_store.similarity_search(query111, n_results=5)

  context = "\n".join([chunk.content for chunk in similar_chunks])

  chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are a financial assistant providing detailed budget summaries based on user data."},
          {"role": "user", "content": f"Here are is the context and the query which is to be answered using the context:\n{context}\n\nNow, {button}"},
      ],
      model="llama-3.3-70b-versatile",
  )

  response111 = chat_completion.choices[0].message.content
  return response111


def EssentialExpenditureAgent(query):
  # query112 = f'Give a detailed description of essential and non-essential expenditures for the month of February.'
  similar_chunks = vector_store.similarity_search(query, n_results=5)

  context = "\n".join([chunk.content for chunk in similar_chunks])

  chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are a financial assistant providing detailed budget summaries based on user data."},
          {"role": "user", "content": f"Here are is the context and the query which is to be answered using the context:\n{context}\n\nNow, {query}. Give a detailed breakdown of essential and non-essential expenditures for the user."},
      ],
      model="llama-3.3-70b-versatile",
  )

  response112 = chat_completion.choices[0].message.content
  return response112


def FinancialAdvisorAgent(context):
  chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are a financial assistant providing detailed budget summaries based on user data."},
          {"role": "user", "content": f'Given context{context}, give a detailed analysis of how can user improve their money saving, do cost-cuttings and improve budget. Also give a monthly or yearly estimate for how much can be saved by better finances'},
      ],
      model="llama-3.3-70b-versatile",
  )

  response113 = chat_completion.choices[0].message.content
  return response113

def FinancialPlanningAgent(query, context):

  chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are a financial assistant providing detailed budget summaries based on user data."},
          {"role": "user", "content": f' You are given a context input {context} and a query {query}, give an estimate to the question, if the user does not improve their budget, and if the user improve their budget. Also give a analysis of the query.'},
      ],
      model="llama-3.3-70b-versatile",
  )

  responselt = chat_completion.choices[0].message.content
  return responselt


def OrchestrationAgent(query):
  chat_completion = client.chat.completions.create(
      messages=[
          {"role": "system", "content": "You are an orchestration agent whose task is to classify queries."},
          {"role": "user", "content": f"""Given user query{query}, classify what does the query ask for from one of the following categories:
          # BudgetBreakDownAgent -- Monthly/yearly/category wise breakdown of budget
          # EssentialExpenditureAgent - Breakdown and analysis of essential and non-essential expenditures
          # FinancialPlanningAgent - Estimate for time needed to achieve a long-term or short-term goal.
          Output format: No explanantion, only a single word answer giving output of the agent to which query was classified to."""},
      ],
      model="llama-3.3-70b-versatile",
  )
  response = chat_completion.choices[0].message.content
  return response


In [ ]:
anvil.server.wait_forever()

Received text: can i buy a car
Classified as: FinancialPlanningAgent
Financial Planning Response: **Analysis of the Query:**

The user is considering purchasing a car, and we need to assess whether they can afford it based on their current financial situation and potential future savings. To estimate the feasibility of buying a car, we'll consider the user's current expenses, potential savings, and the costs associated with owning a car.

**Current Financial Situation:**

* Total Monthly Expenditure: 11500
* Essential Expenditures: 9000
* Non-Essential Expenditures: 2500

**Estimated Car Expenses:**

* Vehicle purchase price: Assume a moderate-priced car costing around 200,000 (this can vary greatly depending on the location, make, and model)
* Monthly car loan payment: Approximately 2500-3500 per month (assuming a 5-year loan with an interest rate of 5-7%)
* Insurance: 1000-2000 per year (approximately 80-170 per month)
* Fuel: 2800 per month (as mentioned in the original data, but th